In [ ]:
# Installation of FEniCS in Colab via
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

### Time dependent Stokes equation ###

In [ ]:
from fenics import *
from mshr import *
from pylab import plt
import numpy as np

# generate domains and meshes

L  = 23.0
H  = 12.0
X  = 3.0
r  = 0.5
nr = 4*16 # 48

rect   = Polygon([Point(0,0),Point(L,0),Point(L,H),Point(0,H)])
disc   = Circle(Point(X,H/2),r,nr)
domain = rect - disc
mesh   = generate_mesh(domain,32)

def create_markers(mesh,X0,xr,yr):
    cell_markers = MeshFunction("bool",mesh,mesh.geometric_dimension())
    for cell in cells(mesh):
        cell_index = cell.index()
        pos = np.reshape(cell.get_vertex_coordinates(),(-1,2))
        xm  = np.mean(pos[:,0])
        ym  = np.mean(pos[:,1])
        rr = ((xm-X0)**2)/(xr*xr) + ((ym-H/2)**2)/(yr*yr)
        if (rr < 1):
            cell_markers[cell_index] = True
        else:
            cell_markers[cell_index] = False
    return cell_markers


mesh = refine(mesh,create_markers(mesh,  12,   10.0, 3.0))
mesh = refine(mesh,create_markers(mesh,   8,    6.0, 1.5))
mesh = refine(mesh,create_markers(mesh,   4,    2.0, 1.0))

plt.figure(figsize=(20,20))
plot(mesh,linewidth=1)
# plt.xlim([0,12])

### Example 4: Convection-diffusion problem ###

We solve the time-dependent problem




\begin{align}
\int_\Omega \partial_t q v\mathrm{d}x + a(q,v)=0
\end{align}

where

$$
a(q,v)=\int_\Omega \alpha\nabla q\cdot\nabla v + (\vec{u}\cdot\nabla q)v\mathrm{d}x
$$

for given functions $\alpha,\vec{u}$.

In [ ]:
# FE definitions
parameters["form_compiler"]["quadrature_degree"] = 3

V    = VectorFunctionSpace(mesh,"CG",1)
W    = FunctionSpace(mesh,"CG",1)

# Define Dirichlet boundary conditions
tol     = 1E-4
def boundary_dirichlet(x, on_boundary):
    return on_boundary and (near(x[1], 0, tol) or near(x[1], H, tol) or near(x[0], 0, tol))

def boundary_disc(x, on_boundary):
    r2 = (x[0]-X)**2 + (x[1]-H/2)**2
    return on_boundary  and near(r2,r*r,0.02)

def boundary_outflow(x, on_boundary):
    return on_boundary and near(x[0], L, tol)


ubc = Expression(("1.0","0"),H=H,degree=3)
bc1 = DirichletBC(V, ubc, boundary_dirichlet)
bc2 = DirichletBC(V, Constant((0.0,0.0)), boundary_disc)
bc  = [bc1,bc2]

bco = DirichletBC(W, Constant(0.0), boundary_outflow)

# single time step
def evolve(old_u, tau):
    
    u,du        = Function(V),TestFunction(V) 
    dudt = (u - old_u)/tau
    Res  = inner(dudt,du)*dx 
    Res += inner(dot(grad(old_u),old_u),du)*dx
    Res += mu*inner(grad(u),grad(du))*dx
    
    u.assign(old_u)
    solve(Res==0, u, bc) 

    return u

# single time step
def chorin_project(old_u):
    
    phi,dphi        = TrialFunction(W),TestFunction(W) 

    a   = inner(grad(phi),grad(dphi))*dx
    L   = div(old_u)*dphi*dx  
    
    phi = Function(W)
    solve(a==L, phi,bco) 
    u   = project(old_u + grad(phi),V)

    return u
    
# main time loop
mu      =  1/150
n_steps =  6000
tau     =  0.01 # 025
t       =  0.0

old_u   = interpolate(Expression(("1.0","0.5*sin(x[0])"),degree=2),V)
file_u  = File("lecture10_u.pvd")
file_w  = File("lecture10_w.pvd")

for i in range(n_steps):
    print(i)
    t = t + tau
    u_int = evolve(old_u, tau)
    u     = chorin_project(u_int)
    
    if ((i % 20)==0):
        w = project(u[1].dx(0)-u[0].dx(1),W)
    
        u.rename("u","")
        w.rename("w","")
        
        file_u << (u,t)
        
    old_u.assign(u)

In [ ]:
from fenics import *
from mshr import *
import numpy as np

# Define 3D geometry
H =  8.0
L = 20.0
r =  0.5
X =  5.0

box    = Box(Point(0, 0, 0),Point(L, H/2, H))
sphere = Sphere(Point(X,0,H/2),r)
domain = box - sphere
mesh   = generate_mesh(domain,24)

def create_markers(mesh,X0,Y0,Z0,xr,yr):
    cell_markers = MeshFunction("bool",mesh,mesh.geometric_dimension())
    for cell in cells(mesh):
        cell_index = cell.index()
        pos = np.reshape(cell.get_vertex_coordinates(),(-1,3))
        xm  = np.mean(pos[:,0])
        ym  = np.mean(pos[:,1])
        zm  = np.mean(pos[:,2])
        rr = ((xm-X0)**2)/(xr*xr) + ((ym-Y0)**2)/(yr*yr) + ((zm-Z0)**2)/(yr*yr)
        if (rr < 1):
            cell_markers[cell_index] = True
        else:
            cell_markers[cell_index] = False
    return cell_markers


mesh = refine(mesh,create_markers(mesh,12,0,H/2, 8.0, 3.0))
mesh = refine(mesh,create_markers(mesh, 8,0,H/2, 4.0, 2.0))
# mesh = refine(mesh,create_markers(mesh, 6,0,H/2, 2.0, 1.0))
#mesh = refine(mesh,create_markers(mesh, 5,0,H/2, 0.7, 0.7))
#mesh = refine(mesh,create_markers(mesh, 5,0,H/2, 0.6, 0.6))

file_m  = File("lecture10_mesh.pvd")
file_m << mesh

In [ ]:
# FE definitions
parameters["form_compiler"]["quadrature_degree"] = 3

V    = VectorFunctionSpace(mesh,"CG",1)
W    = FunctionSpace(mesh,"CG",1)

# Define Dirichlet boundary conditions
tol     = 1E-4
def boundary_dirichlet(x, on_boundary):
    return on_boundary and (near(x[0], 0, tol) or near(x[1],H/2,tol) or near(x[2],0, tol) or near(x[2],H,tol))

def boundary_sym(x, on_boundary):
    return on_boundary and near(x[1], 0, tol)

def boundary_disc(x, on_boundary):
    r2 = (x[0]-X)**2 + x[1]**2 + (x[2]-H/2)**2
    return on_boundary  and near(r2,r*r,0.15)

def boundary_outflow(x, on_boundary):
    return on_boundary and near(x[0], L, tol)


ubc = Expression(("1.0","0","0"),H=H,degree=3)
bc1 = DirichletBC(V       , ubc                    , boundary_dirichlet)
bc2 = DirichletBC(V       , Constant((0.0,0.0,0.0)), boundary_disc)
bc3 = DirichletBC(V.sub(1), Constant(0.0)          , boundary_sym)

bc  = [bc1,bc2,bc3]

bco = DirichletBC(W, Constant(0.0), boundary_outflow)

# single time step
def evolve(old_u, tau):
    
    u,du = TrialFunction(V),TestFunction(V) 
    dudt = u/tau /tau
    a  = inner(u/tau,du)*dx + mu*inner(grad(u),grad(du))*dx
    L  = inner(old_u/tau,du)*dx - inner(dot(grad(old_u),old_u),du)*dx
    u  = Function(V)
    u.assign(old_u)

    solve(a==L,u,bc)
    # Res += 
    # solve(Res==0, u, bc) 

    return u

# single time step
def chorin_project(old_u):
    
    phi,dphi        = TrialFunction(W),TestFunction(W) 

    a   = inner(grad(phi),grad(dphi))*dx
    L   = div(old_u)*dphi*dx  
    
    phi = Function(W)
    solve(a==L, phi,bco) 
    u   = project(old_u + grad(phi),V)

    return u
    
# main time loop
mu      =  1/150
n_steps =  6000
tau     =  0.01*5 # 025
t       =  0.0

old_u   = interpolate(Expression(("1.0","0.5*sin(x[0])","0.5*cos(x[0])"),degree=2),V)
file_u  = File("lecture10_u3d.pvd")
file_w  = File("lecture10_w3d.pvd")

for i in range(n_steps):
    print(i)
    t = t + tau
    print("evolve...")
    u_int = evolve(old_u, tau)
    print("project...")
    u     = chorin_project(u_int)
    print("done.")
    if ((i % 3)==0):
        w = project(curl(u),V)
    
        u.rename("u","")
        w.rename("w","")
        
        file_u << (u,t)
        file_w << (w,t)
        
    old_u.assign(u)